In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [9]:
cap = cv2.VideoCapture('volleyball_match.mp4')
lower_color = np.array([10, 110, 110])
upper_color = np.array([20, 255, 255])
backSub = cv2.createBackgroundSubtractorMOG2(history=3, varThreshold=300)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.bilateralFilter(frame,9,100,75)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask11 = backSub.apply(frame)
    mask11 = cv2.erode(mask11, None,iterations=3)
    mask11 = cv2.dilate(mask11, None,iterations=3)
    mask11 = cv2.GaussianBlur(mask11, (15, 15),0) 
    ret2,mask11 = cv2.threshold(mask11,0,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    res2 = cv2.bitwise_and(frame,frame, mask= mask11)
    a,b,res2 = cv2.split(res2)
    mask = cv2.inRange(hsv, lower_color, upper_color)
    mask = cv2.erode(mask, None,iterations=2)
    mask = cv2.dilate(mask, None,iterations=3)
    mask = cv2.GaussianBlur(mask, (15, 15),0) 
    ret,mask = cv2.threshold(mask,0,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    res = cv2.bitwise_and(frame,frame, mask= mask)
    res = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    res = cv2.bitwise_and(res,res2)
    edges = cv2.Canny(res,150,200)
    res = cv2.cvtColor(res,cv2.COLOR_GRAY2BGR)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    apx=[]
    for cnt in contours:      
        epsilon = 0.02*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)
        cv2.drawContours(res, [approx], -1, (0,0,255), 3)
        apx.append(approx)
    if contours:
        c = max(apx, key=cv2.contourArea)
        M = cv2.moments(c)
        if (M["m00"]==0):
            M["m00"] = 0.0001
        x = int(M["m10"] / M["m00"])
        y = int(M["m01"] / M["m00"])
        cv2.circle(frame, (x, y), 10, (0, 255, 0), -1)
    cv2.imshow("Frame", frame)
    cv2.imshow("mask",mask)
    cv2.imshow("mase",res)
    cv2.imshow("mas",res2)
    cv2.imshow("masey",edges)
    if cv2.waitKey(10) == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()